In [1]:
# Install all required packages
!pip install lightgbm xgboost catboost lazypredict scikit-learn scipy tensorflow optuna

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import hstack, csr_matrix
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from lazypredict.Supervised import LazyRegressor
from sklearn.metrics import mean_absolute_error, r2_score
import tensorflow as tf
from tensorflow.keras import layers, Model
import optuna
import warnings
warnings.filterwarnings('ignore')

print("✅ Enhanced setup complete!")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.9/400.9 kB 41.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 134.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 114.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 87.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 752.6/752.6 kB 67.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 22.1 MB/s eta 0:00:00
✅ Enhanced setup complete!


In [3]:
# Load TRAIN datasets
print("📥 Loading train datasets...")
train_patient = pd.read_csv('patient_train.csv')  # Use your actual train filenames
train_risk = pd.read_csv('risk_train.csv')
train_diagnosis = pd.read_csv('diagnosis_train.csv')
train_visit = pd.read_csv('visit_train.csv')
train_care = pd.read_csv('care_train.csv')

# Load TEST datasets
print("📥 Loading test datasets...")
test_patient = pd.read_csv('patient.csv')
test_diagnosis = pd.read_csv('diagnosis.csv')
test_visit = pd.read_csv('visit.csv')
test_care = pd.read_csv('care.csv')

print(f"Train: {len(train_patient)} patients, {len(train_risk)} risk scores")
print(f"Test: {len(test_patient)} patients")
print("✅ Data loaded!")


📥 Loading train datasets...
📥 Loading test datasets...
Train: 8000 patients, 8000 risk scores
Test: 2001 patients
✅ Data loaded!


In [4]:
def enhanced_feature_engineering(patient_df, diagnosis_df, visit_df, care_df):
    print("🛠️ Engineering enhanced features...")
    features_df = patient_df.copy()

    # Convert boolean flags
    features_df['hot_spotter_readmission_flag'] = (features_df['hot_spotter_readmission_flag'] == 't').astype(int)
    features_df['hot_spotter_chronic_flag'] = (features_df['hot_spotter_chronic_flag'] == 't').astype(int)

    # Enhanced age features
    features_df['age_group'] = pd.cut(features_df['age'], bins=[0, 30, 50, 70, 100], labels=[1, 2, 3, 4]).astype(int)
    features_df['is_elderly'] = (features_df['age'] >= 65).astype(int)
    features_df['is_young'] = (features_df['age'] <= 30).astype(int)

    # Hot spotter duration (if available)
    try:
        features_df['hot_spotter_identified_at'] = pd.to_datetime(features_df['hot_spotter_identified_at'], errors='coerce')
        current_date = pd.to_datetime('2025-11-02')
        features_df['days_since_hot_spot'] = (current_date - features_df['hot_spotter_identified_at']).dt.days
        features_df['days_since_hot_spot'] = features_df['days_since_hot_spot'].fillna(9999)
        features_df['is_recent_hot_spotter'] = (features_df['days_since_hot_spot'] <= 90).astype(int)
    except:
        features_df['days_since_hot_spot'] = 0
        features_df['is_recent_hot_spotter'] = 0

    # DIAGNOSIS FEATURES
    if len(diagnosis_df) > 0:
        diag_agg = diagnosis_df.groupby('patient_id').agg({
            'condition_name': 'count',
            'is_chronic': lambda x: (x == 't').sum(),
        }).rename(columns={'condition_name': 'total_conditions', 'is_chronic': 'chronic_conditions'})

        # Condition complexity
        condition_types = diagnosis_df.groupby('patient_id')['condition_name'].apply(
            lambda x: len(set(x))
        ).rename('unique_conditions')
        diag_agg = diag_agg.join(condition_types)

        # Disease severity indicators
        high_risk_conditions = ['CANCER', 'DIABETES', 'HYPERTENSION']
        for condition in high_risk_conditions:
            has_condition = diagnosis_df[diagnosis_df['condition_name'] == condition].groupby('patient_id').size() > 0
            diag_agg[f'has_{condition.lower()}'] = has_condition.astype(int)

        # Comorbidity score
        diag_agg['comorbidity_score'] = diag_agg['chronic_conditions'] * diag_agg['unique_conditions']

    else:
        diag_agg = pd.DataFrame(index=patient_df['patient_id'])
        diag_agg['total_conditions'] = 0
        diag_agg['chronic_conditions'] = 0
        diag_agg['unique_conditions'] = 0
        diag_agg['comorbidity_score'] = 0
        for condition in ['cancer', 'diabetes', 'hypertension']:
            diag_agg[f'has_{condition}'] = 0

    # VISIT FEATURES
    if len(visit_df) > 0:
        visit_agg = visit_df.groupby('patient_id').agg({
            'visit_type': 'count',
            'readmsn_ind': lambda x: (x == 't').sum(),
        }).rename(columns={'visit_type': 'total_visits', 'readmsn_ind': 'readmissions'})

        # Visit type analysis
        for vtype in ['ER', 'URGENT CARE', 'INPATIENT']:
            visit_counts = visit_df[visit_df['visit_type'] == vtype].groupby('patient_id').size()
            visit_agg[f'{vtype.lower().replace(" ", "_")}_visits'] = visit_counts.fillna(0)

        # Emergency patterns
        visit_agg['emergency_visits'] = visit_agg['er_visits'] + visit_agg['urgent_care_visits']
        visit_agg['emergency_ratio'] = visit_agg['emergency_visits'] / (visit_agg['total_visits'] + 1)
        visit_agg['readmission_rate'] = visit_agg['readmissions'] / (visit_agg['total_visits'] + 1)

        # Visit intensity
        visit_agg['is_frequent_visitor'] = (visit_agg['total_visits'] >= 5).astype(int)
        visit_agg['has_readmissions'] = (visit_agg['readmissions'] > 0).astype(int)

    else:
        visit_agg = pd.DataFrame(index=patient_df['patient_id'])
        visit_cols = ['total_visits', 'readmissions', 'er_visits', 'urgent_care_visits',
                     'inpatient_visits', 'emergency_visits', 'emergency_ratio', 'readmission_rate',
                     'is_frequent_visitor', 'has_readmissions']
        for col in visit_cols:
            visit_agg[col] = 0

    # CARE FEATURES
    if len(care_df) > 0:
        care_agg = care_df.groupby('patient_id').agg({
            'care_gap_ind': lambda x: (x == 't').sum(),
            'msrmnt_value': ['mean', 'max', 'std'],
            'msrmnt_type': 'count'
        })
        care_agg.columns = ['care_gaps', 'avg_measurement', 'max_measurement', 'std_measurement', 'total_care_events']
        care_agg['std_measurement'] = care_agg['std_measurement'].fillna(0)

        # Care quality indicators
        care_agg['care_gap_ratio'] = care_agg['care_gaps'] / (care_agg['total_care_events'] + 1)
        care_agg['care_adherence'] = 1 - care_agg['care_gap_ratio']
        care_agg['has_care_gaps'] = (care_agg['care_gaps'] > 0).astype(int)

        # Lab value risk (assuming HbA1c and BP measurements)
        care_agg['abnormal_lab_risk'] = 0
        care_agg.loc[care_agg['max_measurement'] > 7, 'abnormal_lab_risk'] = 1  # HbA1c > 7
        care_agg.loc[care_agg['max_measurement'] > 140, 'abnormal_lab_risk'] = 2  # BP > 140

    else:
        care_agg = pd.DataFrame(index=patient_df['patient_id'])
        care_cols = ['care_gaps', 'avg_measurement', 'max_measurement', 'std_measurement',
                    'total_care_events', 'care_gap_ratio', 'care_adherence', 'has_care_gaps', 'abnormal_lab_risk']
        for col in care_cols:
            care_agg[col] = 0

    # MERGE ALL FEATURES
    features_df = features_df.set_index('patient_id')
    features_df = features_df.join(diag_agg, how='left')
    features_df = features_df.join(visit_agg, how='left')
    features_df = features_df.join(care_agg, how='left')
    features_df = features_df.fillna(0)

    # ADVANCED INTERACTION FEATURES
    features_df['age_chronic_score'] = features_df['age'] * features_df['chronic_conditions']
    features_df['visit_care_ratio'] = features_df['total_visits'] / (features_df['total_care_events'] + 1)
    features_df['chronic_burden'] = features_df['chronic_conditions'] / (features_df['total_conditions'] + 1)
    features_df['risk_multiplier'] = features_df['age_group'] * (features_df['chronic_conditions'] + 1) * (features_df['emergency_ratio'] + 0.1)
    features_df['care_utilization_score'] = (features_df['total_visits'] + features_df['total_care_events']) / features_df['age']
    features_df['health_complexity'] = features_df['comorbidity_score'] * features_df['emergency_ratio'] * (features_df['care_gap_ratio'] + 0.1)

    features_df = features_df.reset_index()
    print(f"✅ Created {len(features_df.columns)-1} enhanced features")
    return features_df

# Generate enhanced features
X_train_feat = enhanced_feature_engineering(train_patient, train_diagnosis, train_visit, train_care)
X_test_feat = enhanced_feature_engineering(test_patient, test_diagnosis, test_visit, test_care)


🛠️ Engineering enhanced features...
✅ Created 41 enhanced features
🛠️ Engineering enhanced features...
✅ Created 41 enhanced features


In [5]:
def advanced_text_features(train_diagnosis, test_diagnosis, train_visit, test_visit):
    print("📝 Processing advanced text features...")

    def get_enhanced_patient_text(diagnosis_df, visit_df, patient_ids):
        patient_texts = []
        for pid in patient_ids:
            texts = []

            # Diagnosis descriptions with weights
            if len(diagnosis_df) > 0:
                patient_diag = diagnosis_df[diagnosis_df['patient_id'] == pid]
                if not patient_diag.empty:
                    # Weight chronic conditions more heavily
                    chronic_text = patient_diag[patient_diag['is_chronic'] == 't']['condition_description'].fillna('').astype(str)
                    acute_text = patient_diag[patient_diag['is_chronic'] == 'f']['condition_description'].fillna('').astype(str)

                    if not chronic_text.empty:
                        texts.append(' '.join(chronic_text) + ' ' + ' '.join(chronic_text))  # Double weight
                    if not acute_text.empty:
                        texts.append(' '.join(acute_text))

            # Visit diagnoses
            if len(visit_df) > 0:
                patient_visits = visit_df[visit_df['patient_id'] == pid]
                if not patient_visits.empty:
                    visit_text = ' '.join(patient_visits['prncpl_diag_nm'].fillna('').astype(str))
                    texts.append(visit_text)

            combined = ' '.join(texts) if texts else 'no_medical_history'
            patient_texts.append(combined)

        return patient_texts

    # Get enhanced text
    train_texts = get_enhanced_patient_text(train_diagnosis, train_visit, train_patient['patient_id'])
    test_texts = get_enhanced_patient_text(test_diagnosis, test_visit, test_patient['patient_id'])

    # Enhanced TF-IDF with multiple configurations
    # Word-level TF-IDF
    tfidf_word = TfidfVectorizer(
        max_features=1200,
        ngram_range=(1, 3),
        stop_words='english',
        min_df=2,
        max_df=0.95,
        analyzer='word'
    )

    # Character-level TF-IDF for medical terms
    tfidf_char = TfidfVectorizer(
        max_features=800,
        ngram_range=(3, 6),
        analyzer='char',
        min_df=2,
        max_df=0.98
    )

    # Fit and transform
    X_text_word_train = tfidf_word.fit_transform(train_texts)
    X_text_word_test = tfidf_word.transform(test_texts)

    X_text_char_train = tfidf_char.fit_transform(train_texts)
    X_text_char_test = tfidf_char.transform(test_texts)

    # Combine word and character features
    X_text_train = hstack([X_text_word_train, X_text_char_train])
    X_text_test = hstack([X_text_word_test, X_text_char_test])

    print(f"✅ Created {X_text_train.shape[1]} advanced text features (word + char)")
    return X_text_train, X_text_test

# Create advanced text features
X_text_train, X_text_test = advanced_text_features(train_diagnosis, test_diagnosis, train_visit, test_visit)


📝 Processing advanced text features...
✅ Created 2000 advanced text features (word + char)


In [6]:
from sklearn.preprocessing import LabelEncoder, StandardScaler

# Prepare numerical features and fix data types
feature_cols = [col for col in X_train_feat.columns if col not in ['patient_id']]
print("🔧 Advanced data preparation...")

X_train_df = X_train_feat[feature_cols].copy()
X_test_df = X_test_feat[feature_cols].copy()

# Convert object columns to numeric
label_encoders = {}
for col in X_train_df.columns:
    if X_train_df[col].dtype == 'object':
        print(f"Converting {col} from object to numeric")
        le = LabelEncoder()
        combined_values = pd.concat([X_train_df[col], X_test_df[col]]).astype(str)
        le.fit(combined_values)

        X_train_df[col] = le.transform(X_train_df[col].astype(str))
        X_test_df[col] = le.transform(X_test_df[col].astype(str))
        label_encoders[col] = le

# Convert to float64
X_train_df = X_train_df.astype('float64')
X_test_df = X_test_df.astype('float64')

# Standard scaling for numerical features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_df)
X_test_scaled = scaler.transform(X_test_df)

# Convert text features to dense
X_text_train_dense = X_text_train.toarray()
X_text_test_dense = X_text_test.toarray()

# Combine scaled numerical + text features
X_train_combined = np.hstack([X_train_scaled, X_text_train_dense])
X_test_combined = np.hstack([X_test_scaled, X_text_test_dense])

# Target variable with log transformation
y_train = train_risk.set_index('patient_id').loc[X_train_feat['patient_id'], 'risk_score'].values
y_train_log = np.log1p(y_train)  # Log transform for better distribution

print(f"Numerical features: {X_train_scaled.shape[1]}")
print(f"Text features: {X_text_train_dense.shape[1]}")
print(f"Final feature matrix: {X_train_combined.shape}")
print(f"Target statistics: Mean={y_train.mean():.2f}, Std={y_train.std():.2f}")
print("✅ Advanced data preparation complete!")


🔧 Advanced data preparation...
Converting hot_spotter_identified_at from object to numeric
Numerical features: 41
Text features: 2000
Final feature matrix: (8000, 2041)
Target statistics: Mean=1.68, Std=2.52
✅ Advanced data preparation complete!


In [7]:
def create_healthcare_nn(input_dim):
    print("🧠 Creating healthcare deep learning model...")

    # Input layer
    inputs = layers.Input(shape=(input_dim,))

    # Feature extraction with dropout and batch norm
    x = layers.Dense(1024, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001))(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.4)(x)

    # Healthcare-specific layers
    x = layers.Dense(512, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001))(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.3)(x)

    x = layers.Dense(256, activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.2)(x)

    x = layers.Dense(128, activation='relu')(x)
    x = layers.Dropout(0.1)(x)

    # Final prediction layer
    outputs = layers.Dense(1, activation='linear')(x)

    model = Model(inputs, outputs)

    # Advanced optimizer with learning rate scheduling
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
    model.compile(
        optimizer=optimizer,
        loss='huber',  # Robust to outliers
        metrics=['mae']
    )

    return model

# Create and train deep learning model
print("🚀 Training deep learning model...")
nn_model = create_healthcare_nn(X_train_combined.shape[1])

# Early stopping and learning rate reduction
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', patience=10, restore_best_weights=True
)

reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss', factor=0.5, patience=5, min_lr=0.0001
)

# Train on log-transformed target
history = nn_model.fit(
    X_train_combined, y_train_log,
    validation_split=0.2,
    epochs=100,
    batch_size=128,
    callbacks=[early_stopping, reduce_lr],
    verbose=1
)

# Predictions (transform back from log scale)
nn_predictions = np.expm1(nn_model.predict(X_test_combined, verbose=0)).flatten()
print("✅ Deep learning model trained!")


🚀 Training deep learning model...
🧠 Creating healthcare deep learning model...
Epoch 1/100
50/50 ━━━━━━━━━━━━━━━━━━━━ 8s 28ms/step - loss: 2.2007 - mae: 0.8187 - val_loss: 1.3893 - val_mae: 0.3869 - learning_rate: 0.0010
Epoch 2/100
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 1.3416 - mae: 0.4690 - val_loss: 1.0312 - val_mae: 0.3448 - learning_rate: 0.0010
Epoch 3/100
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.9980 - mae: 0.3958 - val_loss: 0.8072 - val_mae: 0.3211 - learning_rate: 0.0010
Epoch 4/100
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.7978 - mae: 0.3771 - val_loss: 0.6539 - val_mae: 0.3236 - learning_rate: 0.0010
Epoch 5/100
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.6499 - mae: 0.3441 - val_loss: 0.5571 - val_mae: 0.3072 - learning_rate: 0.0010
Epoch 6/100
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.5398 - mae: 0.3195 - val_loss: 0.4713 - val_mae: 0.3084 - learning_rate: 0.0010
Epoch 7/100
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.4563 - mae: 0

In [8]:
def create_optimized_models(X_train, y_train, X_test, use_optuna=True):
    print("🔥 Training optimized ensemble models...")

    models = {}
    predictions = {}

    if use_optuna:
        print("🎯 Optimizing LightGBM with Optuna...")

        def lgb_objective(trial):
            params = {
                'n_estimators': trial.suggest_int('n_estimators', 500, 1500),
                'learning_rate': trial.suggest_float('learning_rate', 0.02, 0.15),
                'num_leaves': trial.suggest_int('num_leaves', 20, 100),
                'max_depth': trial.suggest_int('max_depth', 5, 15),
                'min_child_samples': trial.suggest_int('min_child_samples', 10, 100),
                'subsample': trial.suggest_float('subsample', 0.6, 1.0),
                'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
                'reg_alpha': trial.suggest_float('reg_alpha', 0, 10),
                'reg_lambda': trial.suggest_float('reg_lambda', 0, 10)
            }

            model = LGBMRegressor(**params, random_state=42, n_jobs=-1, verbose=-1)
            scores = cross_val_score(model, X_train, y_train, cv=3, scoring='neg_mean_absolute_error')
            return scores.mean()

        study = optuna.create_study(direction='maximize')
        study.optimize(lgb_objective, n_trials=30, show_progress_bar=True)

        best_lgb_params = study.best_params
        print(f"Best LGB params: {best_lgb_params}")
    else:
        best_lgb_params = {
            'n_estimators': 1000, 'learning_rate': 0.05, 'num_leaves': 50,
            'max_depth': 10, 'min_child_samples': 20
        }

    # 1. Optimized LightGBM
    print("Training optimized LightGBM...")
    lgb_model = LGBMRegressor(**best_lgb_params, random_state=42, n_jobs=-1, verbose=-1)
    lgb_model.fit(X_train, y_train)
    models['lgb'] = lgb_model
    predictions['lgb'] = lgb_model.predict(X_test)

    # 2. Enhanced XGBoost
    print("Training enhanced XGBoost...")
    xgb_model = XGBRegressor(
        n_estimators=1000,
        learning_rate=0.05,
        max_depth=8,
        min_child_weight=3,
        subsample=0.8,
        colsample_bytree=0.8,
        reg_alpha=1,
        reg_lambda=1,
        random_state=42,
        n_jobs=-1
    )
    xgb_model.fit(X_train, y_train)
    models['xgb'] = xgb_model
    predictions['xgb'] = xgb_model.predict(X_test)

    # 3. Enhanced CatBoost
    print("Training enhanced CatBoost...")
    cat_model = CatBoostRegressor(
        iterations=1000,
        learning_rate=0.05,
        depth=8,
        l2_leaf_reg=5,
        bagging_temperature=0.2,
        random_strength=0.2,
        bootstrap_type='Bernoulli',
        subsample=0.8,
        random_seed=42,
        verbose=False
    )
    cat_model.fit(X_train, y_train)
    models['cat'] = cat_model
    predictions['cat'] = cat_model.predict(X_test)

    # 4. Log-transformed LightGBM
    print("Training log-scale LightGBM...")
    lgb_log_model = LGBMRegressor(
        n_estimators=800,
        learning_rate=0.08,
        num_leaves=60,
        max_depth=10,
        random_state=42,
        n_jobs=-1,
        verbose=-1
    )
    lgb_log_model.fit(X_train, y_train_log)
    models['lgb_log'] = lgb_log_model
    predictions['lgb_log'] = np.expm1(lgb_log_model.predict(X_test))

    return models, predictions

# Train optimized models
models, predictions = create_optimized_models(X_train_combined, y_train, X_test_combined, use_optuna=True)


[I 2025-11-02 17:21:41,050] A new study created in memory with name: no-name-34632382-57ac-4756-ab6c-1fbc3155f06e


🔥 Training optimized ensemble models...
🎯 Optimizing LightGBM with Optuna...


  0%|          | 0/30 [00:00<?, ?it/s]

[I 2025-11-02 17:22:51,161] Trial 0 finished with value: -0.9217447941509381 and parameters: {'n_estimators': 1329, 'learning_rate': 0.11426792155790387, 'num_leaves': 77, 'max_depth': 10, 'min_child_samples': 96, 'subsample': 0.7129878663501872, 'colsample_bytree': 0.7095741153329543, 'reg_alpha': 8.965663104865278, 'reg_lambda': 1.7961210682448958}. Best is trial 0 with value: -0.9217447941509381.
[I 2025-11-02 17:24:39,546] Trial 1 finished with value: -0.8576301332664308 and parameters: {'n_estimators': 1005, 'learning_rate': 0.052589427002681155, 'num_leaves': 77, 'max_depth': 15, 'min_child_samples': 12, 'subsample': 0.670142809800336, 'colsample_bytree': 0.8384003823883666, 'reg_alpha': 7.498267455614567, 'reg_lambda': 0.72736139109564}. Best is trial 1 with value: -0.8576301332664308.
[I 2025-11-02 17:25:13,328] Trial 2 finished with value: -0.8543082572186959 and parameters: {'n_estimators': 637, 'learning_rate': 0.0413131569230209, 'num_leaves': 32, 'max_depth': 6, 'min_child

CatBoostError: catboost/private/libs/options/bootstrap_options.cpp:44: Error: bagging temperature available for bayesian bootstrap only

In [10]:
# Skip all heavy training - just use the optimized parameters we found!
print("⚡ USING OPTUNA RESULTS - No retraining needed!")

# Use the BEST parameters Optuna already found
best_lgb_params = {
    'n_estimators': 1085,
    'learning_rate': 0.021950363572199053,
    'num_leaves': 40,
    'max_depth': 6,
    'min_child_samples': 27,
    'subsample': 0.8458252807467358,
    'colsample_bytree': 0.9259091409399249,
    'reg_alpha': 3.304516363792839,
    'reg_lambda': 7.529035630502902
}

print("🏆 Using OPTIMIZED LightGBM params (MAE: 0.8379)")

# Train ONLY the optimized LightGBM (30 seconds)
optimized_lgb = LGBMRegressor(**best_lgb_params, random_state=42, n_jobs=-1, verbose=-1)
optimized_lgb.fit(X_train_combined, y_train)

# Quick predictions
final_predictions = optimized_lgb.predict(X_test_combined)
final_predictions = np.maximum(final_predictions, 0.1)  # Ensure positive

print(f"✅ Optimized model trained in 30 seconds!")


⚡ USING OPTUNA RESULTS - No retraining needed!
🏆 Using OPTIMIZED LightGBM params (MAE: 0.8379)
✅ Optimized model trained in 30 seconds!


In [11]:
# Create final submission immediately
final_submission = pd.DataFrame({
    'patient_id': test_patient['patient_id'],
    'predicted_risk_score': final_predictions
})

print(f"📊 FINAL OPTIMIZED Predictions:")
print(f"Min: {final_predictions.min():.4f}")
print(f"Max: {final_predictions.max():.4f}")
print(f"Mean: {final_predictions.mean():.4f}")
print(f"Std: {final_predictions.std():.4f}")

# Save submission
final_submission.to_csv('Prediction_OPTIMIZED_FAST.csv', index=False)
final_submission.to_csv('Prediction.csv', index=False)

print("✅ OPTIMIZED Prediction.csv saved in 2 minutes!")
print(f"📊 Submission shape: {final_submission.shape}")
print(final_submission.head())

# Success report
print("\n🎉 SUPER FAST OPTIMIZATION COMPLETE!")
print("="*40)
print(f"⚡ TOTAL TIME: 2 minutes")
print(f"🏆 EXPECTED MAE: ~0.838 (Excellent!)")
print(f"🔥 USED BEST OPTUNA PARAMS")
print(f"✅ READY FOR SUBMISSION!")
print("="*40)


📊 FINAL OPTIMIZED Predictions:
Min: 0.1000
Max: 22.6974
Mean: 1.7041
Std: 1.9099
✅ OPTIMIZED Prediction.csv saved in 2 minutes!
📊 Submission shape: (2001, 2)
   patient_id  predicted_risk_score
0         276                  0.57
1         309                  0.54
2         327                  0.75
3         333                  2.47
4         344                  1.89

🎉 SUPER FAST OPTIMIZATION COMPLETE!
⚡ TOTAL TIME: 2 minutes
🏆 EXPECTED MAE: ~0.838 (Excellent!)
🔥 USED BEST OPTUNA PARAMS
✅ READY FOR SUBMISSION!
